# Training model

In [5]:
import tensorflow as tf
import os
from keras.utils import img_to_array, load_img
import numpy as np
import cv2 as cv
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

trainPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
testPath = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

batchSize = 64      # Reduce value if you have less GPU

# # Read in example image and get shape
# img = load_img(trainPath + "/Quince/0_100.jpg")
# plt.imshow(img)
# plt.show()

# imgA = img_to_array(img)
# print(imgA.shape)

# Build model
model = Sequential()
model.add(Conv2D(filters=128, kernel_size=3, activation="relu", input_shape=(100,100,3)))
model.add(MaxPooling2D())
model.add(Conv2D(filters=64, kernel_size=3, activation="relu"))
model.add(Conv2D(filters=32, kernel_size=3, activation="relu"))
model.add(MaxPooling2D())
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(5000, activation="relu"))
model.add(Dense(1000, activation="relu"))
model.add(Dense(131, activation="softmax"))

print(model.summary())

# Compile model
model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

# Load data
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.3, horizontal_flip=True, vertical_flip=True, zoom_range=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(trainPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical", shuffle=True)


test_generator = test_datagen.flow_from_directory(testPath, target_size=(100,100), batch_size=batchSize, color_mode="rgb", class_mode="categorical")

stepsPerEpoch = np.ceil(train_generator.samples / batchSize)
validationSteps = np.ceil(test_generator.samples / batchSize)

# Early stopping
stop_early = EarlyStopping(monitor="val_accuracy", patience=5)      # Stop fitting model if it doesn't improve by 5

history = model.fit(train_generator, steps_per_epoch=stepsPerEpoch, epochs=5, validation_data=test_generator, validation_steps=validationSteps, callbacks=[stop_early])

model.save("/kaggle/working/fruits360.h5")     # Add file path to save the model to

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 98, 98, 128)       3584      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 49, 49, 128)      0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 47, 47, 64)        73792     
                                                                 
 conv2d_5 (Conv2D)           (None, 45, 45, 32)        18464     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 22, 22, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 22, 22, 32)       

2023-07-02 22:37:28.201089: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1058/1058 [==============================] - 276s 260ms/step - loss: 3.5241 - accuracy: 0.1491 - val_loss: 1.9349 - val_accuracy: 0.4614
Epoch 2/5
1058/1058 [==============================] - 276s 261ms/step - loss: 1.3912 - accuracy: 0.5662 - val_loss: 0.7682 - val_accuracy: 0.7680
Epoch 3/5
1058/1058 [==============================] - 274s 259ms/step - loss: 0.6346 - accuracy: 0.7964 - val_loss: 0.5061 - val_accuracy: 0.8531
Epoch 4/5
1058/1058 [==============================] - 277s 262ms/step - loss: 0.4012 - accuracy: 0.8809 - val_loss: 0.3317 - val_accuracy: 0.8969
Epoch 5/5
1058/1058 [==============================] - 276s 261ms/step - loss: 0.2214 - accuracy: 0.9300 - val_loss: 0.2843 - val_accuracy: 0.9164


# Testing model

In [27]:
!pip uninstall opencv-python-headless -y 
!pip install opencv-python --upgrade
import tensorflow as tf
import os
from keras.utils import img_to_array, load_img
import numpy as np
import cv2 as cv

# Load model
model = tf.keras.models.load_model("/kaggle/working/fruits360.h5")       # Put in model path here
print(model.summary())

# Load categories
source_folder = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test"
categories = os.listdir(source_folder)
# categories = categories.sort()
print(categories)
print(len(categories))      # Should be 131

# Load and prepare image
def prepareImage(path):
    img = load_img(path, target_size=(100,100))
    imgArray = img_to_array(img)
    # print(imgArray.shape)
    imgArray = np.expand_dims(imgArray, axis=0)
    imgArray = imgArray / 255.
    return imgArray

testImgPath = "/kaggle/input/eggplant/eggplant.jpg"
imageForModel = prepareImage(testImgPath)

resultArray = model.predict(imageForModel, verbose=1)
answers = np.argmax(resultArray, axis=1)
print(categories[answers[0]])

# Show image with text below
img = cv.imread(testImgPath)
cv.putText(img, categories[answers[0]],(0,50), cv.FONT_HERSHEY_COMPLEX, 1, (209, 19, 77), 2)
cv.imshow('img', img)
cv.waitKey(0)




Found existing installation: opencv-python-headless 4.7.0.72
Uninstalling opencv-python-headless-4.7.0.72:
  Successfully uninstalled opencv-python-headless-4.7.0.72
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 98, 98, 128)       3584      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 49, 49, 128)      0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 47, 47, 64)        73792     
                                                                 
 conv2d_5 (Conv2D)           (None, 45, 45, 32)        18464     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 22, 22, 32)       0         
 2D)                

error: OpenCV(4.7.0) /io/opencv/modules/highgui/src/window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
